### 可变数据
摘自 SICP ch2.4

In [1]:
def make_withdraw(balance):
    def withdraw(amount):
        nonlocal balance #声明非局部变量 balance, 这样会向上一层的函数里找 balance
        if amount > balance:
            return 'Insufficient funds'
        balance -= amount
        return balance
    return withdraw

In [2]:
#通过函数 wrapper 我们得到了一个可变对象 wd
wd = make_withdraw(balance = 100)

In [28]:
wd

<function __main__.make_withdraw.<locals>.withdraw(amount)>

In [29]:
#wd接受提款金额返回余额
wd(5)

95

###### 非局部赋值的代价

In [30]:
#wd与wd2指向了同一个函数
wd = make_withdraw(10)
wd2 = wd

In [31]:
print(wd(1))
print(wd2(1))

9
8


In [ ]:
#我们称只含有纯函数的表达式是 引用透明的。
#重新绑定的操作违反了引用透明的条件, 因为他们不仅仅返回了一个值。

#### 示例: 传播约束
<center>我们将实现如下的约束系统</center>

![11](img/constraints.png)
上面的约束系统实现了华氏温度与摄氏温度的互换:<br>
<center>$9 \times c = 5 \times (f - 32)$</center>

In [68]:
#我们首先来实现connecter, 它连接了每个计算模块
def connector(name = None):
    informant = None
    constraints = []

    def set_value(source, value):
        nonlocal informant
        val = connector['val']
        if val is None:
            informant, connector['val'] = source, value
            if name is not None:
                print(name, '=', value)
                print('source is:\n', source)
            inform_all_except(source, 'new_val', constraints)
        else:
            if val != value:
                print('Contradiction detected: ', val, 'vs', value)
    def forget_value(source):
        nonlocal informant
        #print(source, informant)
        if informant == source:
            informant, connector['val'] = None, None
            if name is not None:
                print(name, 'is forgotten.')
            inform_all_except(source, 'forget', constraints)
    
    connector = {
        'val':None,
        'set_val': set_value,
        'forget': forget_value,
        'has_val': lambda: connector['val'] is not None,
        'connect': lambda source: constraints.append(source),
        #print_constraints
        'constraints': lambda: print(constraints)
    }
    return connector

def inform_all_except(source, message, constraints):
    for c in constraints:
        if c!= source:
            c[message]()

In [69]:
#实例化两个连接器 u 和 v
celsius = connector('Celsius')
fahrenheit = connector('Fahrenheit')
#celsius

In [70]:
#我们再来实现两边盒子的基本运算
from operator import add, sub
from operator import mul, truediv
#我们首先实现一个三路限制器,
#这里 a,b,c 是三个连接器,
#ab, ca, cb 是三个函数
def make_ternary_constraint(a, b, c, ab, ca, cb):
    def new_value():
        av, bv, cv = [connector['has_val']() for connector in (a, b, c)]
        #枚举三种情况
        if av and bv:
            c['set_val'](constraint, ab(a['val'], b['val']))
        elif av and cv:
            b['set_val'](constraint, ca(c['val'], a['val']))
        elif bv and cv:
            a['set_val'](constraint, cb(c['val'], b['val']))
    def forget_value():
        for connector in (a, b, c):
            connector['forget'](constraint)
    
    constraint = {'new_val':new_value, 'forget':forget_value}
    for connector in (a, b, c):
        connector['connect'](constraint)
    return constraint
def adder(a, b, c):
    return make_ternary_constraint(a=a, b=b, c=c,
                                   ab=add, ca=sub, cb=sub)
def multiplier(a, b, c):
    return make_ternary_constraint(a, b, c,
                                   mul, truediv, truediv)
def constant(connector, value):
    constraint = {}
    connector['set_val'](constraint, value)
    return constraint

make_ternary_constraint函数实现了connecter之间的互动:
* new_value: 当有一个a, b, c有了新的值的时候, 它会被调用.按照给定的顺序求出第三个值.
* forget_value: 它会清空所有connector的值.
* constraint 是一个调度字典 包含了两个 限制接受的消息.
<br>
我们来看看 make_ternary_constraint 函数怎样被调用的。当它运行时, 会对所有 connector的connect属性初始化:

```python
    for connector in (a, b, c):
        connector['connect'](constraint)
```
回到connector的定义, 这将 constraint加入到 connector函数的constraint列表中.<br>
<br>
在 new_value方法中, 无论哪条分支被执行, 都会调用 connector中的 set_value方法, 再看 set_value方法, 它接收两个参数, 当 connector的 value属性是None时, 分别将 constraint 和 新的值 赋值给 <u>informant</u> 和 value.<br>
<br>
在 forget_value中, 调用所有connector的forget_value方法, connector的forget_value函数会先验证 参数source是否是 informant, 如果是执行清空操作.<br>
<br>
最后, inform_all_except 函数会将 新的数值或数值被清空 广播出去, 对其他的所有的constraint调用相应的方法.<br>
<br>
注意: 每个 make_ternary_constraint 函数对象对应着唯一的 constraint调度字典. 在connector的set_value方法中, 设置了非局部变量 informant的值, 在forget_value中需要验证它的值,
确保他们来自同一个限制.

In [71]:
#最后 我们用 converter方法将网络连接起来
def make_converter(c, f):
    u, v, w, x, y = [connector() for _ in range(5)]
    #左边第一个盒子
    multiplier(c, w, u)
    #中间的盒子
    multiplier(v, x, u)
    #最右的盒子
    adder(v, y, f)
    
    constant(w, 9)
    constant(x, 5)
    constant(y, 32)

make_converter(celsius, fahrenheit)

In [72]:
celsius['constraints']()

[{'new_val': <function make_ternary_constraint.<locals>.new_value at 0x7f7dc08a1268>, 'forget': <function make_ternary_constraint.<locals>.forget_value at 0x7f7dc090c598>}]


In [62]:
#我们('user')将celsius的值设置为25, 它的值会在网络上传播, 于是fahrenheit的也跟着变
celsius['set_val'](source='user', value=25)

Celsius = 25
source is:
 user
Fahrenheit = 77.0
source is:
 {'new_val': <function make_ternary_constraint.<locals>.new_value at 0x7f7dc090d400>, 'forget': <function make_ternary_constraint.<locals>.forget_value at 0x7f7dc090d6a8>}


In [63]:
#我们需要forget_value, 然后在设置值
fahrenheit['set_val'](source='user', value=212)

Contradiction detected:  77.0 vs 212


In [64]:
celsius['forget']('user')

Celsius is forgotten.
Fahrenheit is forgotten.


In [65]:
fahrenheit['set_val'](source='user', value=212)

Fahrenheit = 212
source is:
 user
Celsius = 100.0
source is:
 {'new_val': <function make_ternary_constraint.<locals>.new_value at 0x7f7dc0923268>, 'forget': <function make_ternary_constraint.<locals>.forget_value at 0x7f7dc090de18>}


In [57]:
celsius['forget']('user')

In [66]:
fahrenheit['forget']('user')

Fahrenheit is forgotten.
Celsius is forgotten.
